# Esercitazione 4b - New Language

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

*Consegna*: si richiede un'implementazione di un metodo per la generazione di una nuova lingua (che chiameremo NL). In particolare, partendo da una lingua di partenza L1 (ad es. la lingua Inglese), si prendano i termini di L1 usando un dizionario elettronico (ad es. WordNet o BabelNet). Per ogni termine $t$ ed i suoi sensi $S_t$, dovrete cercare un nuovo termine $tt$ (in una seconda lingua L2 a vostra scelta) da accoppiare a $t$, per la costruzione del termine $t{-}tt$, da inserire in NL. Il termine $tt$ in L2 va selezionato tra quelli meno ambigui per il concetto $S_t$ di riferimento. Si richiede di calcolare un valore di riduzione dell'ambiguità della nuova lingua rispetto a quella di partenza (ad es. calcolando il numero di sensi associabili ai termini $t{-}tt$ in NL rispetto a quelli associabili ai termini $t$ in L1. Una volta implementato il sistema, potrete cambiare la lingua L2 per valutare il potere "disambiguante" di diverse lingue rispetto a quella di partenza L1.


In [18]:
from nltk.corpus import wordnet
import nltk
import sys

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Dizionario L1 e scelta L2

Come risorsa linguistica di riferimento, piuttosto che BabelNet, divenuta una risorsa proprietaria, si è scelto di utilizzare Open Multilingual Wordnet (OMW), un progetto che raccoglie [decine di WordNet nazionali](http://globalwordnet.org/resources/wordnets-in-the-world/) sviluppate nel corso degli anni. OMW è integrato nella libreria NTLK di Python, risultando quindi una scelta particolarmente efficace in relazione allo sviluppo dell'implementazione.

Come L1 si è scelto l'Italiano invece dell'Inglese per utilizzare una WordNet con una dimensione simile alle altre WN nazionali non-inglesi, mentre per L2 si è da subito pensato di utilizzare linguaggi che nel senso comune vengono riconosciuti come particolarmente *specifici* o *precisi*, come il Tedesco, il Latino o una lingua ugrofinnica come l'Ungherese. Le WordNet relative al primo e all'ultimo non sono però integrate in OMW, mentre non si considera il Latino un linguaggio con una copertura adatta al task attuale (verificato empiricamente esplorando *MultiWordNet*). Si è quindi scelto di utulizzare come L2 lo Spagnolo, con cui la verifica di correttezza dell'implementazione è immediata, e il Finlandese (si veda più avanti).

In [19]:
l1_lang = 'ita'
l2_lang = 'spa'
# sorted(wordnet.langs()) # To list available languages

Si definisce un insieme di termini a cui andare ad associare quelli della seconda lingua (come sottolineato a lezione, non si andrà a creare una vera e propria nuova lingua. ma piuttosto un nuovo vocabolario, con maggior potere di disambiguazione). Sarà anche utile memorizzare il numero di sensi totali associati a questo limitato vocabolario, così da poter valutare la performance delle scelte implementative.

In [20]:
l1_dict = ['capo', 'terra', 'scuola', 'porta']

l1_total_senses = sum(len(wordnet.synsets(l1_term, lang=l1_lang)) for l1_term in l1_dict)
print(f"{l1_total_senses} senses for the terms in L1.")

38 senses for the terms in L1.


### Generazione NL

Per comodità si definisce un oggetto per rappresentare un termine in NL, rappresentazione minimale che può essere eventualmente arricchita.

In [21]:
class NL_Term:
  def __init__(self, first_term, second_term):
    self.first_term = first_term
    self.second_term = second_term

  def __str__(self):
    return f"\x1B[3m{self.first_term}-{self.second_term}\x1B[0m"

Seguendo la consegna, per ogni senso $S_t$ si va ad associare al suo termine d'origine $t$ nel dizionario L1 il termine $tt$ *meno abiguo* corrispondente a $S_t$ in L2, ovvero il termine associato a $S_t$ con minor numero di sensi associati totali, andando così a creare un dizionario in NL composto dai termini compositi L1-L2 (se $t$ ha più $n$ sensi associati, si avrà un numero $n$ di $t-tt$ termini).

Bisogna sempre sottolineare la distinzione tra *termine* e *sensi* associati a esso: si partirà infatti da ogni termine in L1, e per ogni senso associato a esso si estrarrano e collezioneranno i termini corrispondenti in L2 (il tutto viene ottenuto con un numero limitato di funzioni di NLTK relative a WN). Una volta ottenuti i termini in L2 si andrà a scegliere (per ogni senso) quello con minor numero di sensi associato. Il risultato saranno quindi tante coppie  *termine*-*termine*, e non *termine*-*senso*, per ogni senso $S_t$ (la cui verbalizzazione è disponibile in L2).

In [22]:
def get_less_ambigous_associated_terms(term, first_lang, second_lang):

  second_lang_terms = {}
  original_term_senses = wordnet.synsets(term, lang=first_lang)

  # For each L1 term's sense collect the terms in L2
  for original_sense in original_term_senses:
    second_lang_terms[original_sense] = []
  for original_sense in original_term_senses:
    second_lang_terms[original_sense].extend(original_sense.lemma_names(second_lang))

  # Trivial search for the least ambiguous term (min # of associated senses) for each sense
  less_ambigous_terms = {}

  for original_sense, terms in second_lang_terms.items():

    less_ambigous_term_senses_number = sys.maxsize # Initialize with Int Max Value

    for term in terms:

      current_term_senses_number = len(wordnet.synsets(term, lang=second_lang))

      if current_term_senses_number < less_ambigous_term_senses_number:
        less_ambigous_terms[original_sense] = term
        less_ambigous_term_senses_number = current_term_senses_number

  return less_ambigous_terms

Si va quindi a creare il nuovo dizionario NL contenente coppie costituite da termini in L1 e i termini associati in L2 *meno ambigui* per ogni senso dei termini in L1.

In [23]:
# Compute NL composite-term for each term in L1
def generate_NL(first_lang_dict, first_lang, second_lang):
  nl_dict = []

  for term in first_lang_dict:
    nl_terms = get_less_ambigous_associated_terms(term, first_lang, second_lang)
    for nl_term in nl_terms.values():
      nl_dict.append(NL_Term(term, nl_term))

  return nl_dict

Si genera NL per Italiano e Spagnolo, per poi compararla con NL generato per Italiano e Finlandese.

In [24]:
nl_dict = generate_NL(l1_dict, l1_lang, l2_lang)

print(f"NL computed over {l1_lang}-{l2_lang} ({len(nl_dict)} terms): ")

for nl_term in nl_dict:
  print(f"  {nl_term} (L1 senses: {len(wordnet.synsets(nl_term.first_term, lang=l1_lang))} / L2 senses: {len(wordnet.synsets(nl_term.second_term, lang=l2_lang))})")

nl_l1_senses = sum(len(wordnet.synsets(nl_term.first_term, lang=l1_lang)) for nl_term in nl_dict)
nl_l2_senses = sum(len(wordnet.synsets(nl_term.second_term, lang=l2_lang)) for nl_term in nl_dict)
print(f"\nAmbiguity reduction: {1 - (nl_l2_senses / nl_l1_senses):0.2f}")

NL computed over ita-spa (27 terms): 
  capo-artículo (L1 senses: 11 / L2 senses: 10)
  capo-primordial (L1 senses: 11 / L2 senses: 6)
  capo-cabeza (L1 senses: 11 / L2 senses: 11)
  capo-cabeza (L1 senses: 11 / L2 senses: 11)
  capo-cabo (L1 senses: 11 / L2 senses: 3)
  capo-cabo (L1 senses: 11 / L2 senses: 3)
  capo-dirigente (L1 senses: 11 / L2 senses: 3)
  capo-jefe (L1 senses: 11 / L2 senses: 6)
  capo-responsable (L1 senses: 11 / L2 senses: 5)
  terra-humanidad (L1 senses: 15 / L2 senses: 3)
  terra-nación (L1 senses: 15 / L2 senses: 4)
  terra-tierra (L1 senses: 15 / L2 senses: 10)
  terra-región (L1 senses: 15 / L2 senses: 4)
  terra-globo (L1 senses: 15 / L2 senses: 4)
  terra-tierra_firme (L1 senses: 15 / L2 senses: 1)
  terra-suelo (L1 senses: 15 / L2 senses: 7)
  terra-finca (L1 senses: 15 / L2 senses: 4)
  terra-tierra (L1 senses: 15 / L2 senses: 10)
  terra-tierra (L1 senses: 15 / L2 senses: 10)
  terra-suelo (L1 senses: 15 / L2 senses: 7)
  scuola-colegio (L1 senses: 6 /

In [25]:
l2_lang = 'fin'

nl_dict = generate_NL(l1_dict, l1_lang, l2_lang)

print(f"NL computed over {l1_lang}-{l2_lang} ({len(nl_dict)} terms): ")

for nl_term in nl_dict:
  print(f"  {nl_term} (L1 senses: {len(wordnet.synsets(nl_term.first_term, lang=l1_lang))} / L2 senses: {len(wordnet.synsets(nl_term.second_term, lang=l2_lang))})")

nl_l1_senses = sum(len(wordnet.synsets(nl_term.first_term, lang=l1_lang)) for nl_term in nl_dict)
nl_l2_senses = sum(len(wordnet.synsets(nl_term.second_term, lang=l2_lang)) for nl_term in nl_dict)
print(f"\nAmbiguity reduction: {1 - (nl_l2_senses / nl_l1_senses):0.2f}")

NL computed over ita-fin (37 terms): 
  capo-kappale (L1 senses: 11 / L2 senses: 10)
  capo-pääasiallinen (L1 senses: 11 / L2 senses: 2)
  capo-pää (L1 senses: 11 / L2 senses: 11)
  capo-säie (L1 senses: 11 / L2 senses: 5)
  capo-nauha (L1 senses: 11 / L2 senses: 11)
  capo-kallo (L1 senses: 11 / L2 senses: 4)
  capo-pää (L1 senses: 11 / L2 senses: 11)
  capo-niemeke (L1 senses: 11 / L2 senses: 2)
  capo-päällikkö (L1 senses: 11 / L2 senses: 6)
  capo-jehu (L1 senses: 11 / L2 senses: 1)
  capo-päällikkö (L1 senses: 11 / L2 senses: 6)
  terra-ihmiskunta (L1 senses: 15 / L2 senses: 1)
  terra-maajohto (L1 senses: 15 / L2 senses: 1)
  terra-valtio (L1 senses: 15 / L2 senses: 3)
  terra-maa (L1 senses: 15 / L2 senses: 17)
  terra-vyöhyke (L1 senses: 15 / L2 senses: 6)
  terra-maapallo (L1 senses: 15 / L2 senses: 4)
  terra-kuiva_maa (L1 senses: 15 / L2 senses: 1)
  terra-kasvumaa (L1 senses: 15 / L2 senses: 1)
  terra-Tellus (L1 senses: 15 / L2 senses: 1)
  terra-maaomaisuus (L1 senses: 15

Si può notare come lo Spagnolo copra solo $27$ dei $38$ sensi totali originali, mentre il Finlandese $37$. Questo può essere dato o da una più grande dimensione della WordNet finlandese rispetto a quella spagnola (un limite quindi della risorsa linguistica) o dalla mancata verbalizzazione di sensi che invece la posseggono in Italiano e in Finlandese. Questa minore o maggiore copertura è una componente che dovrà essere valutata durante la progettazione di uno *score* che descriva il *potere disambiguante* di una lingua L2 rispetto L1.

Più interessante è invece la riduzione dell'ambiguità (il numero di sensi totali di L1 sul numero di sensi totali di L2): il Finlandese non copre solo più sensi dello Spagnolo, ma anche la sua riduzione dell'ambiguità ($57\%$) è maggiore rispetto a quella della seconda lingua ($51\%$). Come tutte le lingue uraliche (comprese quindi le ugrofinniche), il Finlandese è una lingua agglutinante, proprietà evidente dalle *varianti* di *koulu*, e proprio questa proprietà potrebbe essere dietro alla minor ambiguità intrinseca.

È però necessario sviluppare uno *score* che evidenzi questa riduzione in maniera sistematica, per una coppia arbitraria di linguaggi.

### Potere disambiguante

Si introduce uno score per determinare il *potere disambiguante* di una lingua L2 rispetto un'altra L1 (dato il nuovo dizionario NL calcolato). A ogni coppia di linguaggi *L1-L2* (la NL candidata) si associa una tupla composta da:

- Il numero di sensi originali in L1 che sono anche coperti in L2 (*higher is better*).
- La riduzione dell'ambiguità, ovvero la quantità in percentuale dei sensi totali in L2 rispetto a quelli in L1  (*higher is better*).


In [26]:
def compute_disambiguation_power(new_lang_dict, first_lang, second_lang):

  disambiguation_power = (0, 0)

  new_lang_first_senses = sum(len(wordnet.synsets(new_lang_term.first_term, lang=first_lang)) for new_lang_term in new_lang_dict)
  new_lang_second_senses = sum(len(wordnet.synsets(new_lang_term.second_term, lang=second_lang)) for new_lang_term in new_lang_dict)

  senses_coverage = len(new_lang_dict)
  ambiguity_reduction = 1 - (new_lang_second_senses / new_lang_first_senses)

  return (senses_coverage, ambiguity_reduction)

In [27]:
l2_lang = 'spa'
nl_dict = generate_NL(l1_dict, l1_lang, l2_lang)
print(f"{l1_lang}-{l2_lang}: {compute_disambiguation_power(nl_dict, l1_lang, l2_lang)}")

l2_lang = 'fin'
nl_dict = generate_NL(l1_dict, l1_lang, l2_lang)
print(f"{l1_lang}-{l2_lang}: {compute_disambiguation_power(nl_dict, l1_lang, l2_lang)}")

ita-spa: (27, 0.5098039215686274)
ita-fin: (37, 0.56575682382134)


È quindo ora possibile calcolare il potere disambiguante rispetto a qualsiasi coppia di lingue *L1-L2*.

In [28]:
l2_lang = 'jpn'
nl_dict = generate_NL(l1_dict, l1_lang, l2_lang)
print(f"{l1_lang}-{l2_lang}: {compute_disambiguation_power(nl_dict, l1_lang, l2_lang)}")

l2_lang = 'heb'
nl_dict = generate_NL(l1_dict, l1_lang, l2_lang)
print(f"{l1_lang}-{l2_lang}: {compute_disambiguation_power(nl_dict, l1_lang, l2_lang)}")

ita-jpn: (33, 0.8638888888888889)
ita-heb: (8, -6.813953488372093)


Si può notare come il Giapponese copra meno sensi del Finlandese, ma lo faccia in maniera molto meno ambigua (maggiore riduzione dell'ambiguità). È necessario quindi permettere all'utente di esprimere una preferenza sulla copertura e sulla ambiguità volute (si veda dopo).

L'Ebraico ha invece una copertura bassisima e una ambiguità addirittura maggiore di quella di partenza (valore negativo).

### Linguaggio L2 più disambiguante

Ciclando su tutti i linguaggi offerti da OMW è possibile determinare quale sia il linguaggio più disambiguante rispetto il dizionario di partenza.

In [29]:
def compute_langs_with_highest_disambiguation_power(first_lang_dict, first_lang, coverage_cutoff, langs_number, coverage_preference):

  disambiguation_powers = []

  # Compute and store disambiguation powers while computing the highest one
  for second_lang in wordnet.langs():
    if second_lang not in l1_lang:
      new_lang_dict = generate_NL(first_lang_dict, first_lang, second_lang)
      disambiguation_power = compute_disambiguation_power(new_lang_dict, first_lang, second_lang)
      disambiguation_powers.append((second_lang, disambiguation_power[0], disambiguation_power[1]))

  # Filter disambiguation powers selecting only the ones above cutoff
  langs_with_highest_disambiguation_power = [power for power in disambiguation_powers if power[1] > coverage_cutoff]

  if coverage_preference:
    # Sort the remaining disambiguation powers by senses coverage and then by ambiguity reduction
    langs_with_highest_disambiguation_power.sort(key=lambda power: (power[1], power[2]), reverse=True)
  else:
    # Sort the remaining disambiguation powers by ambiguity reduction
    langs_with_highest_disambiguation_power.sort(key=lambda power: power[2], reverse=True)

  return langs_with_highest_disambiguation_power[:langs_number]

In [30]:
# PARAMETERS
cutoff = 6 # Allow sub-optimal original senses' coverage
n_langs = 5 # Desired number of languages (1 for the best one only)
prefer_coverage = False

In [31]:
langs_with_powers = compute_langs_with_highest_disambiguation_power(l1_dict, l1_lang, l1_total_senses - cutoff, n_langs, prefer_coverage)

langs = [lang[0] for lang in langs_with_powers]
powers = [(lang[1], lang[2]) for lang in langs_with_powers]
print(f"{langs} with disambiguation powers {powers}")
print(f"{len(langs)} out of {len(wordnet.langs())} languages available in NLTK WN")

['jpn', 'tha', 'fin', 'zsm', 'slv'] with disambiguation powers [(33, 0.8638888888888889), (35, 0.7900262467191601), (37, 0.56575682382134), (36, 0.5231958762886597), (34, 0.5027624309392265)]
5 out of 32 languages available in NLTK WN


Non si avrà quindi un unico linguaggio con più alto potere disambiguante, ma un numero parametrizzabile dipendente dalla preferenza sulla copertura e sulla riduzione d'ambiguità (per ottenere il singolo linguaggio più disambiguante è semplicemente necessario impostare `n_langs` a $1$).

Bisogna però sottolineare come questo risultato ovviamente dipenda dal dizionario di partenza, come evidende di seguito.

In [32]:
l1_dict = ['capo', 'terra', 'scuola', 'porta', 'spesa', 'combinazione', 'francese']

l1_total_senses = sum(len(wordnet.synsets(l1_term, lang=l1_lang)) for l1_term in l1_dict)
print(f"{l1_total_senses} senses for the terms in L1.")

langs_with_powers = compute_langs_with_highest_disambiguation_power(l1_dict, l1_lang, l1_total_senses - cutoff, n_langs, prefer_coverage)

langs = [lang[0] for lang in langs_with_powers]
powers = [(lang[1], lang[2]) for lang in langs_with_powers]
print(f"{langs} with disambiguation powers {powers}")
print(f"{len(langs)} out of {len(wordnet.langs())} languages available in NLTK WN")

50 senses for the terms in L1.
['fin', 'slv', 'eng', 'ron'] with disambiguation powers [(49, 0.5404814004376368), (46, 0.4903846153846154), (49, 0.4485776805251641), (47, 0.3792325056433409)]
4 out of 32 languages available in NLTK WN


In [33]:
l1_dict = ['salve', 'monte', 'salto', 'fonte']

l1_total_senses = sum(len(wordnet.synsets(l1_term, lang=l1_lang)) for l1_term in l1_dict)
print(f"{l1_total_senses} senses for the terms in L1.")

langs_with_powers = compute_langs_with_highest_disambiguation_power(l1_dict, l1_lang, l1_total_senses - cutoff, n_langs, prefer_coverage)

langs = [lang[0] for lang in langs_with_powers]
powers = [(lang[1], lang[2]) for lang in langs_with_powers]
print(f"{langs} with disambiguation powers {powers}")
print(f"{len(langs)} out of {len(wordnet.langs())} languages available in NLTK WN")

9 senses for the terms in L1.
['nld', 'tha', 'arb', 'jpn', 'als'] with disambiguation powers [(5, 0.7272727272727273), (8, 0.7142857142857143), (5, 0.5882352941176471), (9, 0.5365853658536586), (5, 0.5294117647058824)]
5 out of 32 languages available in NLTK WN
